In [4]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [11]:
import torch
import os
from sentence_transformers import SentenceTransformer
import numpy as np
import openai
from dotenv import load_dotenv

# Carica il modello SBERT pre-addestrato
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
options = options = os.getenv("OPTIONS").split(",")
sentence = "Count me how many truck gold and silver there are."

In [10]:
sentence_embedding = model.encode([sentence])[0]
options_embeddings = model.encode(options)

similarities = {}
for i, option in enumerate(options):
    similarity = np.dot(sentence_embedding, options_embeddings[i]) / (np.linalg.norm(sentence_embedding) * np.linalg.norm(options_embeddings[i]))
    similarities[option] = similarity

best_option = max(similarities, key=similarities.get)

print(f"Most Probability world is '{best_option}' with similarity {similarities[best_option]:.4f}")

Most Probability world is 'cars' with similarity 0.3768


In [ ]:
from openai import OpenAI
import numpy as np

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

dataset = np.array([
    ['red', 5],
    ['blue', 15],
    ['green', 10],
    ['yellow', 12],
    ['white', 20]
])

def get_filter_code(prompt: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=prompt,
        max_tokens=100,
        temperature=0,
    )
    
    return response.choices[0].message.content

In [17]:
query = "Filter all entries with a number greater than 10 without printing them, and print 'hello world'."

dataset = dataset

messages = [
    {"role": "system", "content": "You are an assistant that helps write Python code."},
    {"role": "user", "content": 
    f"""
        Considering the following {dataset} in numpy array format, where the first column is 'color' and the second column is 'number':
        The user's query is: "{query}".

        Write the Python code that filters the dataset based on the query. Do not include comments or import statements, only 
        the Python code.
        """}
]

try:
    filter_code = get_filter_code(prompt=messages).replace("```python", "").replace("```", "").strip()
    exec(filter_code)
except Exception as e:
    print(f"An error occurred: {e}")

hello world
